**Note:** running with on purpose created environment `r5py-env`

In [15]:
import sys
import pandas as pd
import geopandas as gpd
import datetime
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode
sys.argv.append(["--max-memory", "8G"])

points_file = "./../data/gis/tynewear_lsoas_centroids_wgs84.shp"
jobs_file = "./../data/bres_jobs_tynewear.csv"
gs_accesspoints_file = "./../data/gis/tynewear_gs_accesspoints_lsoas.csv"

In [18]:
# reading in the origin/destination points shp (just for tynewear region):
centroids = gpd.read_file(points_file)
centroids['id'] = centroids['LSOA21CD']
centroids.head()

# reading in land use data:
jobs_per_lsoa = pd.read_csv(jobs_file)
jobs_per_lsoa.head()

accesspoints_per_lsoa = pd.read_csv(gs_accesspoints_file)
accesspoints_per_lsoa.head()

AttributeError: 'GeoDataFrame' object has no attribute 'LSOA11CD'

In [8]:
landuse_gdf = pd.merge(centroids,
                       jobs_per_lsoa,
                       how = "outer",
                       left_on = "LSOA21CD",
                       right_on = "LSOA11CD")
landuse_gdf.head()

landuse_gdf = pd.merge(landuse_gdf,
                       accesspoints_per_lsoa,
                       how = "outer",
                       left_on = "LSOA21CD",
                       right_on = "LSOA11CD")
landuse_gdf.head()



,LSOA21CD,geometry,LSOA11CD_x,jobs,LSOA11CD_y,n_access
0,E01008162,POINT (-1.58805 54.95842),E01008162,125.0,E01008162,24.0
1,E01008588,POINT (-1.43999 55.04056),E01008588,450.0,E01008588,0.0
2,E01008510,POINT (-1.49579 54.99911),E01008510,175.0,E01008510,5.0
3,E01008707,POINT (-1.40202 54.90328),E01008707,100.0,E01008707,0.0
4,E01008184,POINT (-1.75247 54.91805),E01008184,300.0,E01008184,10.0


In [11]:
# landuse_gdf.explore("n_access",cmap="Reds", marker_kwds={"radius": 12})

In [13]:
transport_network = TransportNetwork(
    "/Users/azanchetta/OTP/graphs/tynewear15/tyne-and-wear-latest.osm.pbf",
    [
        "/Users/azanchetta/OTP/graphs/tynewear15/itm_north_east_gtfs.zip"
    ]
)

In [17]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=centroids,
    departure=datetime.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK]
)
travel_time_matrix_all = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_all.head()

NoIDColumnError: Origin dataset must contain an 'id' column.